# Import requirements 

In [ ]:
import os
import zipfile
import random

import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import layers
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

# Extract zip files

In [ ]:
dataset_dir = '../input/facial-keypoints-detection'
training_zip = os.path.join(dataset_dir, "training.zip")
test_zip = os.path.join(dataset_dir, "test.zip")

In [ ]:
if not os.path.exists('dataset'):
        os.mkdir('dataset')
        
# Extract training file
with zipfile.ZipFile(os.path.join(dataset_dir, "training.zip")) as zipref:
    zipref.extractall('dataset')
    
# Extract test file
with zipfile.ZipFile(os.path.join(dataset_dir, "test.zip")) as zipref:
    zipref.extractall('dataset')

# Examine data values

In [ ]:
df = pd.read_csv("dataset/training.csv")

##### Simple check to see if there any missing values in training dataset

In [ ]:
# Check if there is missing values in data
isMissing = False
for i in df.index:
    if (df.iloc[i].isnull().values.any()):
        isMissing = True
        break
print(isMissing)

##### A large part of the training dataset has missing value in it. So we can't drop all those data.

In [ ]:
# Drop where row has nan value
dropped_df = df.dropna(axis='index')

# Visualize for comparision
ypos = df.index
title = ['before', 'after']

values = [df.index.size, dropped_df.index.size]
plt.bar(title, values)

# Examine images

In [ ]:
def load_img(s):
    # Convert image to PIL Image format
    s = [int(num) for num in s.split(' ')]
    im = np.array(s)
    im = im.reshape((96, 96, 1))
    im = np.concatenate([im, im, im], axis=-1)
    return tf.keras.preprocessing.image.array_to_img(im)

#### From 10 random images, i find that all keypoints in the dataset should be in the same area. So we should be fine when using mean to generate missing values

In [ ]:
im_index = random.choices(df.index, k=10)
im_list = [load_img(df.loc[index]['Image']) for index in im_index]

fig, ax = plt.subplots(nrows=2, ncols=5, figsize=(20, 5))
index = 0
for r, row in enumerate(ax):
    for c, col in enumerate(row):
        ax[r][c].imshow(im_list[index], cmap='gray')
        ax[r][c].set_xticks([])
        ax[r][c].set_yticks([])
        ax[r][c].set_title(im_index[index])
        index += 1

In [ ]:
# Replace missing values with mean
df_mean = df.mean()
processed_df = df.fillna(df_mean)

#### See the results after replace nan with mean

In [ ]:
missing_index = []
for i in df.index:
    if (df.iloc[i].isnull().values.any()):
        missing_index.append(i)

In [ ]:
def draw_keypoints_on_image(df):
    img = load_img(df[-1])
    img = keras.preprocessing.image.array_to_img(img)
    draw = ImageDraw.Draw(img)
    
    values = np.array(df[:-1], dtype=np.int32).reshape((-1, 2))
    
    x_values = values[:, 0]
    y_values = values[:, 1]
    
    size = 1
    for x, y in zip(x_values, y_values):
        draw.ellipse((x-size, y-size, x+size, y+size), fill = 'red', outline ='red')
        
    return img

In [ ]:
im_index = random.choices(missing_index, k=5)
im_list = [draw_keypoints_on_image(processed_df.loc[index]) for index in im_index]

fig, ax = plt.subplots(nrows=2, ncols=5, figsize=(20, 5))

index = 0
for c in range(5):
    origin_im = load_img(df.loc[index]['Image'])
    processed_im = draw_keypoints_on_image(processed_df.loc[index])
    for r in range(2):
        if (r == 0):
            ax[r][c].imshow(np.asarray(origin_im))
        else:
            ax[r][c].imshow(np.asarray(processed_im))
            
        ax[r][c].set_xticks([])
        ax[r][c].set_yticks([])
        ax[r][c].set_title(im_index[index])
    index += 1

# Prepare data

In [ ]:
train_labels = processed_df.drop("Image", axis=1)

train_images = processed_df['Image']
train_images = np.array([keras.preprocessing.image.img_to_array(load_img(im)) for im in train_images])

# Create simple baseline model

In [ ]:
model = keras.Sequential([
    layers.Input(shape=(96, 96, 3)),
    layers.LayerNormalization(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPool2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPool2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPool2D(),
    layers.Flatten(),
    layers.Dense(30)
])
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])
model.summary()

In [ ]:
model.fit(train_images, train_labels, 
          epochs=10,
          batch_size=32, 
          validation_split=0.2)

# Prepare test data

In [ ]:
test_df = pd.read_csv('dataset/test.csv')
test_images = np.array([np.asarray(load_img(test_df.loc[i]['Image'])) for i in range(test_df.shape[0])])

In [ ]:
predicts = model.predict(test_images)

In [ ]:
sample_test = random.randint(0, test_images.shape[0])
x_test = []
y_test = []
for i, num in enumerate(predicts[sample_test]):
    if(i % 2 == 0):
        x_test.append(num)
    else:
        y_test.append(num)

#### Let's see how the modle perform

In [ ]:
plt.imshow(test_images[sample_test])
plt.scatter(x_test, y_test, color='r')